In [2]:
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.compute as pc
import h5py
from src.generators import *
import time
from matplotlib import pyplot as plt
np.random.seed(0)

In [3]:
text = "Sono rosso"
bb_class = np.random.randint(0,9)

bb1 = np.array([0.5,0.5,0.5,0.5])  #YOLO format
                                    #Expected conversion to the other format: [0.25,0.25,0.75,0.75]

bb2 = np.array((0.0977, 0.2604, 0.8789, 0.7812)) #NOT YOLO format (xmin,ymin,xmax,ymax)
                                    #Expected conversion to YOLO format: [0.4883, 0.5208, 0.7812, 0.5208]

im = np.random.rand(3,125,125)
im_b = im.tobytes()
im_class = np.random.randint(0,9)
shape = (3,125,125)

sample1 = {
                    "image_feature":[{
                        "image" : im_b,
                        "shape" : shape,
                        "class_feature" : [
                            {
                                "label" : im_class
                            }
                        ],
                        "bounding_box_feature":[{
                            "bbox" : bb1,
                            "text_feature":[{
                                "description" : text
                            }]
                        },
                        {
                            "bbox" : bb2,
                            "class_feature":[
                                {
                                    "label" : bb_class
                                }
                            ]
                        }]
                    }
                    ]
                }

In [5]:
text1 = "Foto di una nebulosa"
text2 = "Foto infrarossi di una nebulosa"
image1 = np.random.rand(3,125,125)
image2 = np.random.rand(3,125,125)
image3 = np.random.rand(3,125,125)
shape = (3,125,125)


bb1 = np.random.rand(4)
bb2 = np.random.rand(4)
bb3 = np.random.rand(4)
bb4 = np.random.rand(4)
bb5 = np.random.rand(4)
image4 = np.random.rand(3,125,125)
label1 = np.random.randint(100,size=1)[0]

im1b = image1.tobytes()
im1b = image1.tobytes()
im1b = image1.tobytes()
im1b = image1.tobytes()

# print(np.frombuffer(im1b.as_buffer(), dtype=np.float32).shape)

sample2 = {
    "image_feature": [
        {
            "image" : image1.tobytes(),
            "shape" : shape,
            "boundingbox_feature":[{
                "bbox" : bb1,
                "image_1_feature": [{
                    "image" : image4.tobytes(),
                    "shape" : shape,
                }]
            },
            {
                "bbox" : bb2
            },
            {
                "bbox" : bb3,
                "label_feature": [{
                    "label" : label1
                }]
            }]
        },
        {
            "image" : image2.tobytes(),
            "shape" : shape,
            "boundingbox_feature":[{
                "bbox" : bb4
            },{
                "bbox" : bb5
            }]
        },
        {
            "image" : image3.tobytes(),
            "shape" : shape,
            "text_1_feature":[
                {
                    "text": text2
                }
            ]
        }
    ],
    "text_feature" : [
        {
            "text": text1
        }
    ],

}

In [24]:
samples = [sample2,sample1]
table = pa.Table.from_pylist(samples)

In [29]:
len(table)

2

In [28]:
table.take([1,1])

pyarrow.Table
image_feature: list<item: struct<bounding_box_feature: list<item: struct<bbox: list<item: double>, class_feature: list<item: struct<label: int64>>, text_feature: list<item: struct<description: string>>>>, boundingbox_feature: list<item: struct<bbox: list<item: double>, image_1_feature: list<item: struct<image: binary, shape: list<item: int64>>>, label_feature: list<item: struct<label: int32>>>>, class_feature: list<item: struct<label: int64>>, image: binary, shape: list<item: int64>, text_1_feature: list<item: struct<text: string>>>>
  child 0, item: struct<bounding_box_feature: list<item: struct<bbox: list<item: double>, class_feature: list<item: struct<label: int64>>, text_feature: list<item: struct<description: string>>>>, boundingbox_feature: list<item: struct<bbox: list<item: double>, image_1_feature: list<item: struct<image: binary, shape: list<item: int64>>>, label_feature: list<item: struct<label: int32>>>>, class_feature: list<item: struct<label: int64>>, image: 

In [17]:
len(table.column("image_feature").chunk(0).values.field("boundingbox_feature").values.field("image_1_feature").values.field("image").to_pylist())
table.column("image_feature").chunk(0).values.field("boundingbox_feature").values.field("image_1_feature").values.field("image").to_pylist()

-- is_valid: all not null
-- child 0 type: binary
  [
    ...
  ]
-- child 1 type: list<item: int64>
  [
    [
      3,
      125,
      125
    ]
  ]

In [10]:
len(table.column("image_feature").chunk(0).values)

4